# The Battle of Neighborhoods - segmenting and clustering magical Kenya's sceneries

### Install necessary libraries

In [1]:
!conda install -c conda-forge geopy --yes # to determine latitude and longitude
!conda install -c conda-forge folium=0.5.0 --yes # to render maps

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

### Import necessary libraries

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim 
import folium
import numpy as np
import json # library to handle JSON files
import requests # library to handle requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_rows', None)

### Read in Kenyan neighborhoods

In [9]:
df = pd.read_csv('kenya_neighborhoods.csv', header=0)
df.head()

,Neighborhood
0,Mombasa
1,Nairobi
2,Kisumu
3,Diani
4,Kilifi


### Let us see the size of the read data

In [14]:
df.shape

(88, 1)

### Are there any duplicate neighborhood names?

In [84]:
len(df['Neighborhood'].unique())

88

The unique neighborhoods are 88, same as the dataframe size, so all are unique

### Create dataframe to hold the neighborhood name, latitude and longitude data

In [23]:
column_names =['Neighborhood','Latitude','Longitude']
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Neighborhood,Latitude,Longitude


In [24]:
neighborhood_only_df = df['Neighborhood']
neighborhood_only_df.head()

0    Mombasa
1    Nairobi
2     Kisumu
3      Diani
4     Kilifi
Name: Neighborhood, dtype: object

### Define function to get neighborhood's latitude and longitude from Geopy

In [25]:
def Get_Coordinates(neighborhoods_list):
    global neighborhoods
    for neighborhood in neighborhoods_list:             
        try:
            #get latitude and longitude for neighborhood using Geopy
            address = neighborhood + ", KE"
            geolocator = Nominatim(user_agent="ken_explorer")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            print('The geograpical coordinates of {} are {}, {}.'.format(neighborhood, latitude, longitude))      
            
            neighborhoods = neighborhoods.append({'Neighborhood': neighborhood,
                                  'Latitude':latitude,
                                  'Longitude':longitude},ignore_index=True)      
        except:
            print('Coordinates for {} were not found'.format(neighborhood))
            neighborhoods = neighborhoods.append({'Neighborhood': neighborhood,
                                  'Latitude': np.nan,
                                  'Longitude': np.nan},ignore_index=True)
            
            

### Get the coordinates for the neighborhoods and store in a dataframe

In [ ]:
Get_Coordinates(neighborhood_only_df)

In [20]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Mombasa,-4.039015,39.648391
1,Nairobi,-1.283253,36.817245
2,Kisumu,-0.102911,34.754176
3,Diani,-4.290581,39.585806
4,Kilifi,-3.150739,39.675072


In [42]:
neighborhoods.dropna(inplace=True)

In [44]:
neighborhoods.shape

(78, 3)

### Draw map of Kenya and add markers for each town using folium

In [37]:
#get latitude and longitude for Kenya using Geopy

address = 'Kenya, KE'
geolocator = Nominatim(user_agent="ken_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Kenya are {}, {}.'.format(latitude, longitude))

map_kenya = folium.Map(location=[latitude, longitude], zoom_start=10)
#map_kenya

The geograpical coordinates of Kenya are -0.023559, 37.9061928.


In [45]:
# add markers to the Kenya map for each neighborhood and draw the map

for lat, lng,neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kenya)  
    
map_kenya

### Set parameters for Foursquare

In [54]:
CLIENT_ID = 'ZLBZ2XERFFWYQLPGZACQ2GQMXEXTOAUGBTRF4AGH3NE21WEI' # your Foursquare ID
CLIENT_SECRET = 'SER2GAHGZG2GOFYTQISZC1JGVRMI45UCYD0U5TZMX0FBRSGY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
RADIUS = 10000 # define radius. The maximum supported radius is currently 100,000 meters

### Define a function to get nearby venues from Foursquare

In [55]:
def getNearbyVenues(neighborhoods_list, latitudes, longitudes):
    
    venues_list=[]
    for neighborhood, lat, lng in zip(neighborhoods_list, latitudes, longitudes):
        print(neighborhood)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get the venues around each neighborhood from Foursquare

In [56]:
kenya_venues = getNearbyVenues(neighborhoods_list=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Mombasa
Nairobi
Kisumu
Diani
Kilifi
Kwale
Lamu
Malindi
Masai Mara
Naivasha
Nakuru
Nanyuki
Narok
Naromoru
Tsavo
Wasini
Watamu
Aberdare
Amboseli
Arabuko Sokoke
Central Island
Chyulu Hills
Hell's Gate
Lake Nakuru
Meru
Mombasa Marine Park
Mount Elgon National Park
Mount Kenya National Park
Mount Longonot
Nairobi National Park
Ol Donyo Sabuk
Ruma
Saiwa Swamp
Sibiloi
Tsavo East
Tsavo West
Bisanadi
Boni
Buffalo Springs
Dodori
Kakamega Forest
Kisumu Impala
Lake Bogoria
Mwea
Samburu
Shimba Hills
Witu Forest
Kiunga
Tana River
Baringo
Bogoria
Chala
Elmenteita
Jipe
Kamnarok
Kenyatta
Magadi
Lake Naivasha
Turkana
Lake Victoria
Bamburi
Funzi Island
Mtwapa
Tiwi Beach
Watamu Beach
Che Shale
Kikambala Beach
Dunga Beach
Turtle Bay
Shanzu
Vipingo Beach
Gazi
Galu
Kinondo beach
Chale Island
Msambweni beach
Shimoni
Wasini Island beach


### Below is the size of the dataframe of the venues

In [57]:
kenya_venues.shape

(4483, 7)

### Were all towns returned?

In [58]:
print('Neighborhoods with returned venues are ', len(kenya_venues['Neighborhood'].unique()))

Neighborhoods with returned venues are  78


So, all the neighborhoods were returned with their venues

### Below is the dataframe with the venues returned by Foursquare

In [59]:
kenya_venues.head(2)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mombasa,-4.039015,39.648391,Cafesserie - Maison de qualitè,-4.019841,39.720781,Café
1,Mombasa,-4.039015,39.648391,Tamarind Dhow,-4.049301,39.681443,Seafood Restaurant


### Let us see the venues returned for one neighborhood

In [60]:
kenya_venues[kenya_venues['Neighborhood']=='Malindi'].reset_index(drop=True)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malindi,-3.216599,40.116593,Karen Brixen Cafe Malindi,-3.209104,40.117532,Café
1,Malindi,-3.216599,40.116593,Medina Palms,-3.361686,40.003833,Beach
2,Malindi,-3.216599,40.116593,"Osteria del Chianti, Malindi",-3.219514,40.125946,Italian Restaurant
3,Malindi,-3.216599,40.116593,Malindi Marine National Park,-3.254740,40.131856,Beach
4,Malindi,-3.216599,40.116593,Simba Dishes Malindi,-3.217145,40.118823,African Restaurant
5,Malindi,-3.216599,40.116593,Stardust,-3.209317,40.117013,Nightclub
6,Malindi,-3.216599,40.116593,Come Back Disco Lounge Bar Watamu,-3.354508,40.019880,Nightclub
7,Malindi,-3.216599,40.116593,Ocean Sports Watamu,-3.361121,40.008041,Resort
8,Malindi,-3.216599,40.116593,Pata Pata Beach Club,-3.197697,40.121690,Cocktail Bar
9,Malindi,-3.216599,40.116593,Che Shale,-3.064106,40.173055,Hotel


### We see the venues that were returned per neighborhood

In [61]:
kenya_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aberdare,100,100,100,100,100,100
Amboseli,37,37,37,37,37,37
Arabuko Sokoke,68,68,68,68,68,68
Bamburi,90,90,90,90,90,90
Baringo,25,25,25,25,25,25
Bisanadi,4,4,4,4,4,4
Bogoria,63,63,63,63,63,63
Boni,100,100,100,100,100,100
Buffalo Springs,51,51,51,51,51,51


### How many unique categories can be curated from all the returned venues

In [62]:
print('There are {} uniques venue categories.'.format(len(kenya_venues['Venue Category'].unique())))

There are 194 uniques venue categories.


# We analyze each town

In [63]:
# one hot encoding
kenya_onehot = pd.get_dummies(kenya_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kenya_onehot['Neighborhood'] = kenya_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kenya_onehot.columns[-1]] + list(kenya_onehot.columns[:-1])
kenya_onehot = kenya_onehot[fixed_columns]

kenya_onehot.head()

,Zoo Exhibit,ATM,African Restaurant,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
kenya_onehot.shape

(4483, 194)

### Group rows by neighborhood and take the mean of the frequency of occurrence of each venue category per neighborhood

In [65]:
kenya_grouped = kenya_onehot.groupby('Neighborhood').mean().reset_index()
kenya_grouped

,Neighborhood,Zoo Exhibit,ATM,African Restaurant,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,Aberdare,0.00,0.000000,0.060000,0.000000,0.00,0.000000,0.00,0.00,0.00,...,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Amboseli,0.00,0.027027,0.027027,0.027027,0.00,0.000000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Arabuko Sokoke,0.00,0.044118,0.029412,0.014706,0.00,0.000000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Bamburi,0.00,0.044444,0.033333,0.011111,0.00,0.000000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Baringo,0.00,0.000000,0.080000,0.000000,0.00,0.000000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Bisanadi,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Bogoria,0.00,0.000000,0.095238,0.000000,0.00,0.000000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Boni,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.01
8,Buffalo Springs,0.00,0.000000,0.000000,0.000000,0.00,0.019608,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Central Island,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Here is the size of the grouped dataframe

In [79]:
kenya_grouped.shape

(78, 194)

### Let us see the 3 commonest/top venues per neighborhood

In [67]:
num_top_venues = 3

for neighborhood in kenya_grouped['Neighborhood']:
    print("----"+neighborhood+"----")
    temp = kenya_grouped[kenya_grouped['Neighborhood'] == neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aberdare----
    venue  freq
0   Hotel  0.15
1  Resort  0.10
2    Café  0.07


----Amboseli----
    venue  freq
0   Hotel  0.30
1  Resort  0.08
2    Park  0.08


----Arabuko Sokoke----
    venue  freq
0   Hotel  0.13
1  Resort  0.12
2   Beach  0.09


----Bamburi----
    venue  freq
0  Resort  0.13
1   Beach  0.10
2   Hotel  0.09


----Baringo----
                venue  freq
0               Hotel  0.16
1              Lounge  0.12
2  African Restaurant  0.08


----Bisanadi----
        venue  freq
0  Restaurant  0.25
1      Resort  0.25
2   BBQ Joint  0.25


----Bogoria----
                venue  freq
0               Hotel  0.19
1  African Restaurant  0.10
2           Nightclub  0.08


----Boni----
                  venue  freq
0                 Beach  0.14
1                Resort  0.11
2  Caribbean Restaurant  0.10


----Buffalo Springs----
                  venue  freq
0  Fast Food Restaurant  0.18
1           Gas Station  0.12
2           Pizza Place  0.12


----Central Island----


### Create a dataframe of top 10 venues per town

Define a function to sort the venues in descending order

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe with the top 10 venues for each town.

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = kenya_grouped['Neighborhood']

for ind in np.arange(kenya_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kenya_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aberdare,Hotel,Resort,Lounge,Café,African Restaurant,Shopping Mall,Restaurant,Bar,Coffee Shop,BBQ Joint
1,Amboseli,Hotel,Resort,Café,Park,Scenic Lookout,Mountain,Italian Restaurant,Tourist Information Center,Campground,RV Park
2,Arabuko Sokoke,Hotel,Resort,Beach,Italian Restaurant,Seafood Restaurant,ATM,Nightclub,Café,Restaurant,Cocktail Bar
3,Bamburi,Resort,Beach,Hotel,Italian Restaurant,Seafood Restaurant,ATM,African Restaurant,Golf Course,Nightclub,Restaurant
4,Baringo,Hotel,Lounge,African Restaurant,Indian Restaurant,Nightclub,Bed & Breakfast,Hotel Bar,Ice Cream Shop,Pizza Place,Department Store


# Cluster the towns of Kenya using k-means algorithm

In [91]:
# set number of clusters to be 4
kclusters = 4

kenya_grouped_clustering = kenya_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kenya_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 0, 0, 2, 2, 2, 0, 2, 2], dtype=int32)

### Create a new dataframe that includes the cluster and the top 10 venues for each town

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

kenya_merged = neighborhoods

# merge kenya_grouped with towns to add latitude/longitude for each town
kenya_merged = kenya_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

kenya_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mombasa,-4.039015,39.648391,0,Resort,Hotel,Seafood Restaurant,Beach,Italian Restaurant,ATM,Golf Course,African Restaurant,Scenic Lookout,Coffee Shop
1,Nairobi,-1.283253,36.817245,2,Hotel,Café,Lounge,African Restaurant,Bar,Shopping Mall,Coffee Shop,Restaurant,Italian Restaurant,Bistro
2,Kisumu,-0.102911,34.754176,2,Shopping Mall,Hotel,Lounge,Nightclub,Department Store,Restaurant,Bed & Breakfast,Food,Bar,Cocktail Bar
3,Diani,-4.290581,39.585806,0,Seafood Restaurant,Resort,Beach,Hotel,Italian Restaurant,Golf Course,Café,African Restaurant,Scenic Lookout,Coffee Shop
4,Kilifi,-3.150739,39.675072,0,Hotel,Resort,Italian Restaurant,Beach,Seafood Restaurant,Nightclub,Café,Restaurant,Golf Course,Coffee Shop


In [93]:
kenya_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mombasa,-4.039015,39.648391,0,Resort,Hotel,Seafood Restaurant,Beach,Italian Restaurant,ATM,Golf Course,African Restaurant,Scenic Lookout,Coffee Shop
1,Nairobi,-1.283253,36.817245,2,Hotel,Café,Lounge,African Restaurant,Bar,Shopping Mall,Coffee Shop,Restaurant,Italian Restaurant,Bistro
2,Kisumu,-0.102911,34.754176,2,Shopping Mall,Hotel,Lounge,Nightclub,Department Store,Restaurant,Bed & Breakfast,Food,Bar,Cocktail Bar
3,Diani,-4.290581,39.585806,0,Seafood Restaurant,Resort,Beach,Hotel,Italian Restaurant,Golf Course,Café,African Restaurant,Scenic Lookout,Coffee Shop
4,Kilifi,-3.150739,39.675072,0,Hotel,Resort,Italian Restaurant,Beach,Seafood Restaurant,Nightclub,Café,Restaurant,Golf Course,Coffee Shop
5,Kwale,-4.183607,39.105096,0,Seafood Restaurant,Hotel,Resort,Beach,Italian Restaurant,Golf Course,African Restaurant,Café,Scenic Lookout,Coffee Shop
6,Lamu,-2.284674,40.864033,1,Hotel,Resort,Boat or Ferry,Airport,Farm,French Restaurant,Forest,Football Stadium,Food Court,Food
7,Malindi,-3.216599,40.116593,0,Resort,Hotel,Italian Restaurant,Beach,Seafood Restaurant,Cocktail Bar,Coffee Shop,Nightclub,Hotel Bar,Café
8,Masai Mara,-1.285418,36.853290,2,Hotel,Café,Lounge,Coffee Shop,Shopping Mall,African Restaurant,Bar,Restaurant,Bistro,Italian Restaurant
9,Naivasha,-0.721480,36.438738,2,Hotel,Resort,Lounge,Café,African Restaurant,Shopping Mall,Restaurant,Bar,BBQ Joint,Italian Restaurant


### Visualize the resulting clusters

In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kenya_merged['Latitude'], kenya_merged['Longitude'], kenya_merged['Neighborhood'], kenya_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Have a look at each of the clusters

Cluster 1

In [95]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 0, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mombasa,Resort,Hotel,Seafood Restaurant,Beach,Italian Restaurant,ATM,Golf Course,African Restaurant,Scenic Lookout,Coffee Shop
3,Diani,Seafood Restaurant,Resort,Beach,Hotel,Italian Restaurant,Golf Course,Café,African Restaurant,Scenic Lookout,Coffee Shop
4,Kilifi,Hotel,Resort,Italian Restaurant,Beach,Seafood Restaurant,Nightclub,Café,Restaurant,Golf Course,Coffee Shop
5,Kwale,Seafood Restaurant,Hotel,Resort,Beach,Italian Restaurant,Golf Course,African Restaurant,Café,Scenic Lookout,Coffee Shop
7,Malindi,Resort,Hotel,Italian Restaurant,Beach,Seafood Restaurant,Cocktail Bar,Coffee Shop,Nightclub,Hotel Bar,Café
15,Wasini,Seafood Restaurant,Hotel,Beach,Resort,Italian Restaurant,Café,African Restaurant,Restaurant,Coffee Shop,Boat or Ferry
16,Watamu,Hotel,Resort,Beach,Italian Restaurant,Seafood Restaurant,ATM,Nightclub,Café,Golf Course,African Restaurant
19,Arabuko Sokoke,Hotel,Resort,Beach,Italian Restaurant,Seafood Restaurant,ATM,Nightclub,Café,Restaurant,Cocktail Bar
27,Mombasa Marine Park,Resort,Beach,Italian Restaurant,Seafood Restaurant,Hotel,ATM,Nightclub,Café,African Restaurant,Golf Course
33,Ruma,Beach,Café,Hotel,German Restaurant,Park,Burger Joint,Italian Restaurant,Seafood Restaurant,Ice Cream Shop,Bar


Cluster 2

In [96]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 1, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Lamu,Hotel,Resort,Boat or Ferry,Airport,Farm,French Restaurant,Forest,Football Stadium,Food Court,Food
14,Tsavo,Hotel,Scenic Lookout,Train Station,History Museum,Park,Resort,Rest Area,Restaurant,Campground,Bus Station
18,Amboseli,Hotel,Resort,Café,Park,Scenic Lookout,Mountain,Italian Restaurant,Tourist Information Center,Campground,RV Park
21,Chyulu Hills,Hotel,Café,National Park,Mountain,Italian Restaurant,Resort,Rest Area,Park,Coffee Shop,Scenic Lookout
36,Tsavo East,Hotel,Scenic Lookout,Train Station,History Museum,Park,Resort,Rest Area,Restaurant,Campground,Bus Station
37,Tsavo West,Hotel,Scenic Lookout,Train Station,History Museum,Park,Resort,Rest Area,Restaurant,Campground,Bus Station
52,Witu Forest,Hotel,Resort,Boat or Ferry,Airport,Farm,French Restaurant,Forest,Football Stadium,Food Court,Food
54,Kiunga,Hotel,ATM,Resort,Airport,Boat or Ferry,Farm,French Restaurant,Forest,Football Stadium,Food Court
58,Chala,Hotel,Café,Resort,Scenic Lookout,Mountain,National Park,Park,Campground,Coffee Shop,Restaurant
61,Jipe,Hotel,Café,Scenic Lookout,Mountain,Resort,Indian Restaurant,Italian Restaurant,African Restaurant,Grocery Store,Campground


Cluster 3

In [97]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 2, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Nairobi,Hotel,Café,Lounge,African Restaurant,Bar,Shopping Mall,Coffee Shop,Restaurant,Italian Restaurant,Bistro
2,Kisumu,Shopping Mall,Hotel,Lounge,Nightclub,Department Store,Restaurant,Bed & Breakfast,Food,Bar,Cocktail Bar
8,Masai Mara,Hotel,Café,Lounge,Coffee Shop,Shopping Mall,African Restaurant,Bar,Restaurant,Bistro,Italian Restaurant
9,Naivasha,Hotel,Resort,Lounge,Café,African Restaurant,Shopping Mall,Restaurant,Bar,BBQ Joint,Italian Restaurant
10,Nakuru,Resort,Hotel,African Restaurant,Coffee Shop,Café,Scenic Lookout,Gift Shop,Fast Food Restaurant,National Park,Nightclub
11,Nanyuki,Hotel,Resort,Lounge,Shopping Mall,Coffee Shop,Nightclub,Hotel Bar,BBQ Joint,African Restaurant,Grocery Store
12,Narok,Resort,National Park,Hotel,Building,Airport,Forest,Flea Market,Coffee Shop,Scenic Lookout,Café
13,Naromoru,Resort,Hotel,Nightclub,Lounge,African Restaurant,Shopping Mall,Café,Restaurant,Hotel Bar,Golf Course
17,Aberdare,Hotel,Resort,Lounge,Café,African Restaurant,Shopping Mall,Restaurant,Bar,Coffee Shop,BBQ Joint
20,Central Island,Border Crossing,Shopping Mall,BBQ Joint,Lake,Discount Store,Distillery,French Restaurant,Forest,Football Stadium,Food Court


Cluster 4

In [98]:
kenya_merged.loc[kenya_merged['Cluster Labels'] == 3, kenya_merged.columns[[0] + list(range(4, kenya_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,Tana River,Motel,Pet Store,Zoo,Falafel Restaurant,Forest,Football Stadium,Food Court,Food,Flea Market,Fast Food Restaurant


### Concluding on the clusters generated 

Cluster 1 consists of a lot of seafood restaurants and beaches. These towns are placed right next to the ocean.

Cluster 2 consisted of hotels, scenic lookouts, mountains, forests and parks.

Cluster 3 consists of hotels, shopping malls, lounges, night clubs and national parks.

Kenya is a beautiful country consisting of beaches, nationals parks and all the other basic ammenities like hotels and shopping malls

# The End